# Project: Poetry generation

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# import string
import requests
import pandas as pd
import numpy as np


import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Conv1D, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
try:
    response = requests.get('https://raw.githubusercontent.com/laxmimerit/poetry-data/master/adele.txt')

    print('Status: ', response.status_code)
    print('[INFO] Data have been downloaded successfully')
except Exception as e: 
    print('[ERROR] Data were not downloaded')
    print(e)

# In case the data cannot be downloaded, manually load file 'adele.txt' contained in directory Data

Status:  200
[INFO] Data have been downloaded successfully


In [3]:
# Split text to lines
data = response.text.splitlines()

## Build LSTM Model

In [4]:
# Tokenization
token = Tokenizer()
token.fit_on_texts(data)

# Encoded text
encoded_text = token.texts_to_sequences(data)

# Calculate vocabulary size
vocab_size = len(token.word_counts) + 1
print('[INFO] Vocabulary size: ', vocab_size)


[INFO] Vocabulary size:  1396


##  Prepare Training Data

In [5]:
# Prepare training data
datalist = []
for d in encoded_text:
  if len(d)>1:
    for i in range(2, len(d)):
      datalist.append(d[:i])
      
      
# Padding
max_length = 20
sequences = pad_sequences(datalist, maxlen=max_length, padding='pre')


# Create inputs/outputs
X = sequences[:, :-1]
y = sequences[:, -1]

y = to_categorical(y, num_classes=vocab_size)


seq_length = X.shape[1]
print('[INFO] Sequence length: ', seq_length)

[INFO] Sequence length:  19


## LSTM Model Training

In [6]:
# model = Sequential()
# model.add(Embedding(vocab_size, 50, input_length=seq_length))
# model.add(LSTM(100, return_sequences=True))
# model.add(LSTM(100))
# model.add(Dense(100, activation='relu'))
# model.add(Dense(vocab_size, activation='softmax'))

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(Conv1D(64, activation='relu', kernel_size=4, strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

In [7]:
score = model.fit(X, y,
                  batch_size=32, 
                  verbose=True, 
                  epochs=100)


Epoch 1/100
445/445 [==============================] - 2s 4ms/step - loss: 5.6419 - accuracy: 0.0424
Epoch 2/100
445/445 [==============================] - 2s 4ms/step - loss: 5.1054 - accuracy: 0.0661
Epoch 3/100
445/445 [==============================] - 2s 4ms/step - loss: 4.6580 - accuracy: 0.1166
Epoch 4/100
445/445 [==============================] - 2s 4ms/step - loss: 4.1836 - accuracy: 0.1838
Epoch 5/100
445/445 [==============================] - 2s 4ms/step - loss: 3.7200 - accuracy: 0.2642
Epoch 6/100
445/445 [==============================] - 2s 4ms/step - loss: 3.2932 - accuracy: 0.3357
Epoch 7/100
445/445 [==============================] - 2s 4ms/step - loss: 2.9022 - accuracy: 0.3956
Epoch 8/100
445/445 [==============================] - 2s 4ms/step - loss: 2.5618 - accuracy: 0.4515
Epoch 9/100
445/445 [==============================] - 2s 4ms/step - loss: 2.2628 - accuracy: 0.5054
Epoch 10/100
445/445 [==============================] - 2s 5ms/step - loss: 2.0123 - accura

## Poetry Generation

In [8]:
def generate_poetry(seed_text = '', n_lines = 5, poetry_length = 10):

  for i in range(n_lines):
    text = []
    for _ in range(poetry_length):
      encoded = token.texts_to_sequences([seed_text])
      encoded = pad_sequences(encoded, maxlen=seq_length, padding='pre')

      y_pred = np.argmax(model.predict(encoded), axis=-1)

      predicted_word = ""
      for word, index in token.word_index.items():
        if index == y_pred:
          predicted_word = word
          break

      seed_text = seed_text + ' ' + predicted_word
      text.append(predicted_word)

    seed_text = text[-1]
    text = ' '.join(text)
    print(text)


    
seed_text = 'i love you'
generate_poetry(seed_text, 5)

when the rain is blowing in your right things i
know it ain't easy giving up your mind some i'll
be better to you let me stay here for just
look and you will know i re love again i
know it ain't easy giving up your mind some i'll
